In [ ]:
from importlib import reload
import dataprep_functions as f

In [39]:
f = reload(f)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [3]:
census_df = f.merge_census(f.census_1, f.census_2)

In [4]:
f.community_detection_iters()

new_york
mob
0
1
2
3
4
5
6
7
8
9
new_york
fol_hh
0
1
2
3
4
5
6
7
8
9
los_angeles
mob
0


/home/barcsab/urban_communities/scripts/dataprep_functions.py:338: RuntimeWarning: invalid value encountered in true_divide
  det = det / normadet


1
2
3
4
5
6
7
8
9
los_angeles
fol_hh
0
1
2
3
4
5
6
7
8
9


In [5]:
f.calc_consen()

new_york
mob
ms
new_york
mob
mgn
new_york
fol_hh
ms
new_york
fol_hh
mgn
los_angeles
mob
ms
los_angeles
mob
mgn
los_angeles
fol_hh
ms
los_angeles
fol_hh
mgn


In [6]:
tract_df = f.create_tract_df_with_degree()

In [7]:
tract_df, community_df = f.create_community_df(tract_df)

In [8]:
community_df, com_mod_df = f.calc_community_modularity(community_df, tract_df)

geoid
14000US36005021800    0
14000US36005026500    0
14000US36047054400    1
14000US36119002202    0
14000US36119006801    0
                     ..
14000US36047040500    0
14000US36047056100    0
14000US36081032700    0
14000US36047049800    0
14000US36047051900    0
Name: S, Length: 4603, dtype: int64
0
{0, 1, 2, 3}
[0, 1, 2, 3]
geoid
14000US36005021800    0
14000US36005026500    0
14000US36047016000    0
14000US36047054400    0
14000US36047039100    0
                     ..
14000US36059410000    3
14000US36081059200    3
14000US34003032202    2
14000US34029727001    5
14000US34029731202    5
Name: S, Length: 4603, dtype: int64
0
{0, 1, 2, 3, 4, 5}
[0, 1, 2, 3, 4, 5]
geoid
14000US34017007100    0
14000US34017013800    0
14000US34003043001    0
14000US34029735103    0
14000US34003047400    0
                     ..
14000US36081086300    1
14000US36081105900    2
14000US36047025400    2
14000US34019011800    4
14000US34019011900    4
Name: S, Length: 4389, dtype: int64
0
{0, 1, 2, 3,

In [52]:
network_df = f.calc_modularity(com_mod_df)

In [10]:
community_df = f.calc_modularity_contribution(community_df, network_df)

In [53]:
community_df, network_df = f.calc_tract_number(tract_df, community_df, network_df, f.cbsacode)

In [57]:
# EZ HIBAS!!!






network_df = f.calc_network_density(network_df)

In [31]:
network_df = f.calc_community_county_similarity(tract_df, network_df)

In [32]:
tract_df, census_df = f.clean_census_to_tract_df(tract_df, census_df)

In [33]:
community_df, network_df = f.calc_sum_and_std(tract_df, community_df, network_df)

In [44]:
tract_df, community_df = f.calc_income_percentile(tract_df, community_df)

In [45]:
tract_df = f.create_poor_dummy(tract_df)

In [48]:
tract_df = f.diff_from_network_avg(tract_df, network_df) # TODO

In [49]:
tract_df.head(2).T

,0,1
geoid,14000US36005021800,14000US36005021800
S,0,0
city,new_york,new_york
algorithm_type,ms,ms
g_type,mob,mob
degree,22,22
degreecent,0.000133472,0.000133472
ones,1,1
county,005,005
population_1,6207,6207


In [50]:
community_df.head(2).T

,0,1
city,los_angeles,los_angeles
algorithm_type,mgn,mgn
g_type,fol_hh,fol_hh
S,0,0
degree_avg,34.1102,34.1102
degreecent_avg,0.000270154,0.000270154
modularity_S,0.073805,0.073805
modularity,0.411042,0.411042
mod_S_p,0.179556,0.179556
tract_sum,626,626


In [51]:
network_df.head(2).T

,0,1
city,los_angeles,los_angeles
algorithm_type,mgn,mgn
g_type,fol_hh,mob
modularity,0.411042,0.508086
tract_sum,2842,2895
tract_city_sum,2890,2890
nmi_to_counties,0.275325,0.42906
adj_nmi_to_counties,0.274756,0.428683
adj_rand_to_counties,0.118033,0.204907
population_sum_1,12690179,12844018


## Save tables

In [ ]:
tract_df.to_csv('../data/tract_df')
community_df.to_csv('../data/community_df')
network_df.to_csv('../data/network_df')

In [ ]:
# tisztitás

In [2]:
%matplotlib inline



# import matplotlib.pyplot as plt


#import seaborn as sns
#import networkx as nx

# import scipy
import csv

import matplotlib.cm as cm


import community as community_louvain
from copy import deepcopy

from itertools import product
import networkx.algorithms.community as nx_comm
from scipy.spatial.distance import pdist, squareform

#from shapely.geometry import Point, LineString
from geopandas import GeoDataFrame

#import matplotlib.lines as lines

In [ ]:
# kiszámolom a S_cons-t és ki fájlba

In [ ]:
beolvasom

In [ ]:
hozzáad

ezt is kiment

In [ ]:
cbsacode = pd.read_csv("../data/cbsacode_shortname_tracts.csv",sep=";", index_col=0)

cbsacode['clean_name'] = cbsacode["short_name"].map(lambda s: s.split("/")[0].replace(' ','_').replace('.','').lower())

In [ ]:
# KIMENŐ EZ LESZ MAJD, CSAK HIBA NÉLÜLI


# Properties of tracts

tract_df = pd.read_csv('../data/network_properties.csv')


In [ ]:
# calculate proportions - mean tract population is not the same among communities

# for 2012
tract_df['educ_bach_p_1'] = tract_df['education_bachelor_1']/tract_df['population_1']
tract_df['educ_total_p_1'] = tract_df['education_total_1']/tract_df['population_1']
tract_df['white_p_1'] = tract_df['white_1']/tract_df['population_1']
tract_df['black_p_1'] = tract_df['black_1']/tract_df['population_1']
tract_df['native_p_1'] = tract_df['native_1']/tract_df['population_1']
tract_df['asian_p_1'] = tract_df['asian_1']/tract_df['population_1']

# for 2017
tract_df['educ_bach_p_2'] = tract_df['education_bachelor_2']/tract_df['population_2']
tract_df['educ_total_p_2'] = tract_df['education_total_2']/tract_df['population_2']
tract_df['white_p_2'] = tract_df['white_2']/tract_df['population_2']
tract_df['black_p_2'] = tract_df['black_2']/tract_df['population_2']
tract_df['native_p_2'] = tract_df['native_2']/tract_df['population_2']
tract_df['asian_p_2'] = tract_df['asian_2']/tract_df['population_2']

In [ ]:
tract_once = tract_df[tract_df['algorithm_type'] == 'mg']
network_df = deepcopy(tract_once.groupby(['city','g_type'])['educ_bach_p_1','white_p_1','black_p_1','native_p_1','asian_p_1', 'income_1', 
                                                          'educ_bach_p_2','white_p_2','black_p_2','native_p_2','asian_p_2', 'income_2'].mean().reset_index())

# every tract is counted twice - once for each algorithm_type -- solved
network_df = network_df.rename(columns = {'educ_bach_p_1' : 'educ_bach_p_avg_1','white_p_1' : 'white_p_avg_1', 'black_p_1' : 'black_p_avg_1', 'native_p_1' : 'native_p_avg_1', 'asian_p_1' : 'asian_p_avg_1', 'income_1' : 'income_avg_1',
                                          'educ_bach_p_2' : 'educ_bach_p_avg_2','white_p_2' : 'white_p_avg_2', 'black_p_2' : 'black_p_avg_2', 'native_p_2' : 'native_p_avg_2', 'asian_p_2' : 'asian_p_avg_2', 'income_2' : 'income_avg_2'})